In [1]:
from huggingface_hub import notebook_login
# notebook_login()

*Model Loading*


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
    

# Input query
query = "What is phase diagram"

# Tokenize the input
inputs = tokenizer(query, return_tensors="pt")

# Generate response
output_tokens = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=model.config.eos_token_id,
    do_sample=True,
    top_k=500,
    top_p=0.95,
)

# Decode and print the response
response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(response)


What is phase diagram and do I see myself doing?

With Phase D you can get both them all at once


*Data Loading*

In [2]:
# Read the text file

with open("data.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [3]:

# Read the text file
with open("data.txt", "r", encoding="utf-8") as file:
    text = file.read()
# Split the text into chunks
tokenizer.pad_token = tokenizer.eos_token
from torch.nn.utils.rnn import pad_sequence
import torch

def split_text_into_chunks(text, tokenizer, max_length):
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=False)
    tokens_input_ids = tokens["input_ids"][0]
    tokens_att_masks = tokens["attention_mask"][0]

    # Split into chunks of `max_length`
    input_ids_chunks = [tokens_input_ids[i:i + max_length] for i in range(0, len(tokens_input_ids), max_length)]
    attention_mask_chunks = [tokens_att_masks[i:i + max_length] for i in range(0, len(tokens_att_masks), max_length)]

    # Pad the last chunk if it is shorter than `max_length`
    if len(input_ids_chunks[-1]) < max_length:
        padding_length = max_length - len(input_ids_chunks[-1])
        input_ids_chunks[-1] = torch.cat(
            [input_ids_chunks[-1], torch.full((padding_length,), tokenizer.pad_token_id)]
        )
        attention_mask_chunks[-1] = torch.cat(
            [attention_mask_chunks[-1], torch.zeros(padding_length, dtype=torch.long)]
        )

    # Create the dataset dictionary
    train_dataset = {
        'input_ids': input_ids_chunks,
        'attention_mask': attention_mask_chunks,
        'labels': input_ids_chunks,  # Labels for causal language modeling
    }

    return train_dataset


# Tokenize and split into chunks
train_dataset = split_text_into_chunks(text, tokenizer, 1024)


Token indices sequence length is longer than the specified maximum sequence length for this model (138104 > 1024). Running this sequence through the model will result in indexing errors


In [4]:
print(len(train_dataset["attention_mask"]))
for i, val in enumerate(train_dataset['input_ids']):
    if len(val) != 1024 : 
        print ("input_ids", i, len(val))
        print(tokenizer.decode(val, skip_special_tokens=True))
for i, val in enumerate(train_dataset['attention_mask']):
    if len(val) != 1024 : print ("attention mask", i, len(val))


135


Training

In [5]:

from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from datasets import Dataset


# Apply LoRA configuration
lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # Task type for GPT-like models
    inference_mode=False,
    r=2,  # Low-rank size
    lora_alpha=4,
    fan_in_fan_out=True, # Scaling factor
    lora_dropout=0.4,  # Dropout for LoRA layers
)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
          output_dir=r"\result",
          per_device_train_batch_size=2,
          num_train_epochs=1,
          fp16=True, 
      )

trainer = Trainer(
model,
train_dataset=Dataset.from_dict(train_dataset),
args=training_args,
)
trainer.train()
trainer.save_model()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


  0%|          | 0/68 [00:00<?, ?it/s]

{'train_runtime': 426.097, 'train_samples_per_second': 0.317, 'train_steps_per_second': 0.16, 'train_loss': 4.296060001148897, 'epoch': 1.0}


In [7]:
trainer.save_model(r"C:/Users/hiran/Desktop/nlp/result")

In [8]:
model.save_pretrained(r"C:/Users/hiran/Desktop/nlp/finalmodel")
tokenizer.save_pretrained(r"C:/Users/hiran/Desktop/nlp/finaltokens")

('C:/Users/hiran/Desktop/nlp/finaltokens\\tokenizer_config.json',
 'C:/Users/hiran/Desktop/nlp/finaltokens\\special_tokens_map.json',
 'C:/Users/hiran/Desktop/nlp/finaltokens\\vocab.json',
 'C:/Users/hiran/Desktop/nlp/finaltokens\\merges.txt',
 'C:/Users/hiran/Desktop/nlp/finaltokens\\added_tokens.json',
 'C:/Users/hiran/Desktop/nlp/finaltokens\\tokenizer.json')

In [ ]:
# Push to Hugging Face Hub
model_name = "GPT2finetuned"
